In [18]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re
from nltk.probability import FreqDist
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [19]:
df1=pd.read_csv("../../bugfix-commits-subject-fx.csv")

In [20]:
df1

,commit,bug_id,subject,churn
0,e413c3c4d11d93aca4b022ad9b4a1dea8b432af5,1542883,"Bug 1542883, Remove unused wpt manifest files,...",671424
1,a1b638d149015419baa713063f4a5f7874837986,1381842,Bug 1381842 - Update web-platform-tests to rev...,521797
2,1bf284ae7396bf9f95440c85123382571a5c7385,1419296,Bug 1419296 - Update web-platform-tests to fa2...,399007
3,6892237b54272a2938d9f14824a9207563c2f7ae,1478457,Bug 1478457 [wpt PR 12185] - Fix manifest path...,383419
4,96d519e918295f080f3963e132b8be0a2fbdf5c6,449443,Bug 449443 - Upgrade to SQLite 3.6.4 This chan...,176333
...,...,...,...,...
626296,5676846b3b61a99790169a2ed23abb76f38639b1,798951,Bug 798951 - Intermittent browser_webconsole_b...,1
626297,56901d3ac8fcd3219a65790edfa504a1272d2129,1389010,"Bug 1389010, part 7 - Call AllBordersSameWidth...",1
626298,1ee861335f5f0c1e70925943d0520f5ceeacc956,326972,Use nsContentUtils helpers for some localizing...,1
626299,875f559c288abd157c6fd59d7deb2af1d2658426,809245,Bug 809245 - add preliminary touch support to ...,1


In [21]:
import string


string.punctuation

# creating a function
def remove_punctuation(txt):
    txt_nopunt = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunt

# Assuming 'df1' is a valid DataFrame
df1['msg_clean'] = df1['subject'].apply(lambda x: remove_punctuation(x))
df1

,commit,bug_id,subject,churn,msg_clean
0,e413c3c4d11d93aca4b022ad9b4a1dea8b432af5,1542883,"Bug 1542883, Remove unused wpt manifest files,...",671424,Bug 1542883 Remove unused wpt manifest files r...
1,a1b638d149015419baa713063f4a5f7874837986,1381842,Bug 1381842 - Update web-platform-tests to rev...,521797,Bug 1381842 Update webplatformtests to revisi...
2,1bf284ae7396bf9f95440c85123382571a5c7385,1419296,Bug 1419296 - Update web-platform-tests to fa2...,399007,Bug 1419296 Update webplatformtests to fa27b7...
3,6892237b54272a2938d9f14824a9207563c2f7ae,1478457,Bug 1478457 [wpt PR 12185] - Fix manifest path...,383419,Bug 1478457 wpt PR 12185 Fix manifest paths o...
4,96d519e918295f080f3963e132b8be0a2fbdf5c6,449443,Bug 449443 - Upgrade to SQLite 3.6.4 This chan...,176333,Bug 449443 Upgrade to SQLite 364 This changes...
...,...,...,...,...,...
626296,5676846b3b61a99790169a2ed23abb76f38639b1,798951,Bug 798951 - Intermittent browser_webconsole_b...,1,Bug 798951 Intermittent browserwebconsolebug7...
626297,56901d3ac8fcd3219a65790edfa504a1272d2129,1389010,"Bug 1389010, part 7 - Call AllBordersSameWidth...",1,Bug 1389010 part 7 Call AllBordersSameWidth i...
626298,1ee861335f5f0c1e70925943d0520f5ceeacc956,326972,Use nsContentUtils helpers for some localizing...,1,Use nsContentUtils helpers for some localizing...
626299,875f559c288abd157c6fd59d7deb2af1d2658426,809245,Bug 809245 - add preliminary touch support to ...,1,Bug 809245 add preliminary touch support to m...


In [15]:
#tokenizing the msg_clean

import re

def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens

df1['summary_tokenized']=df1['msg_clean'].apply(lambda x: tokenize(x.lower()))

df1

,commit,bug_id,subject,churn,msg_clean,summary_tokenized
0,e413c3c4d11d93aca4b022ad9b4a1dea8b432af5,1542883,"Bug 1542883, Remove unused wpt manifest files,...",671424,Bug 1542883 Remove unused wpt manifest files r...,"[bug, 1542883, remove, unused, wpt, manifest, ..."
1,a1b638d149015419baa713063f4a5f7874837986,1381842,Bug 1381842 - Update web-platform-tests to rev...,521797,Bug 1381842 Update webplatformtests to revisi...,"[bug, 1381842, update, webplatformtests, to, r..."
2,1bf284ae7396bf9f95440c85123382571a5c7385,1419296,Bug 1419296 - Update web-platform-tests to fa2...,399007,Bug 1419296 Update webplatformtests to fa27b7...,"[bug, 1419296, update, webplatformtests, to, f..."
3,6892237b54272a2938d9f14824a9207563c2f7ae,1478457,Bug 1478457 [wpt PR 12185] - Fix manifest path...,383419,Bug 1478457 wpt PR 12185 Fix manifest paths o...,"[bug, 1478457, wpt, pr, 12185, fix, manifest, ..."
4,96d519e918295f080f3963e132b8be0a2fbdf5c6,449443,Bug 449443 - Upgrade to SQLite 3.6.4 This chan...,176333,Bug 449443 Upgrade to SQLite 364 This changes...,"[bug, 449443, upgrade, to, sqlite, 364, this, ..."
...,...,...,...,...,...,...
626296,5676846b3b61a99790169a2ed23abb76f38639b1,798951,Bug 798951 - Intermittent browser_webconsole_b...,1,Bug 798951 Intermittent browserwebconsolebug7...,"[bug, 798951, intermittent, browserwebconsoleb..."
626297,56901d3ac8fcd3219a65790edfa504a1272d2129,1389010,"Bug 1389010, part 7 - Call AllBordersSameWidth...",1,Bug 1389010 part 7 Call AllBordersSameWidth i...,"[bug, 1389010, part, 7, call, allborderssamewi..."
626298,1ee861335f5f0c1e70925943d0520f5ceeacc956,326972,Use nsContentUtils helpers for some localizing...,1,Use nsContentUtils helpers for some localizing...,"[use, nscontentutils, helpers, for, some, loca..."
626299,875f559c288abd157c6fd59d7deb2af1d2658426,809245,Bug 809245 - add preliminary touch support to ...,1,Bug 809245 add preliminary touch support to m...,"[bug, 809245, add, preliminary, touch, support..."


In [16]:
#importing nltk to remove stopwords from the summary_tokenized

import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

def remove_stopwords(txt_tokenized):
    txt_clean=[word for word in txt_tokenized if word not in stopwords]
    return txt_clean


df1['no_swds']=df1['summary_tokenized'].apply(lambda x: remove_stopwords(x))
df1

[nltk_data] Downloading package stopwords to /home/taj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,commit,bug_id,subject,churn,msg_clean,summary_tokenized,no_swds
0,e413c3c4d11d93aca4b022ad9b4a1dea8b432af5,1542883,"Bug 1542883, Remove unused wpt manifest files,...",671424,Bug 1542883 Remove unused wpt manifest files r...,"[bug, 1542883, remove, unused, wpt, manifest, ...","[bug, 1542883, remove, unused, wpt, manifest, ..."
1,a1b638d149015419baa713063f4a5f7874837986,1381842,Bug 1381842 - Update web-platform-tests to rev...,521797,Bug 1381842 Update webplatformtests to revisi...,"[bug, 1381842, update, webplatformtests, to, r...","[bug, 1381842, update, webplatformtests, revis..."
2,1bf284ae7396bf9f95440c85123382571a5c7385,1419296,Bug 1419296 - Update web-platform-tests to fa2...,399007,Bug 1419296 Update webplatformtests to fa27b7...,"[bug, 1419296, update, webplatformtests, to, f...","[bug, 1419296, update, webplatformtests, fa27b..."
3,6892237b54272a2938d9f14824a9207563c2f7ae,1478457,Bug 1478457 [wpt PR 12185] - Fix manifest path...,383419,Bug 1478457 wpt PR 12185 Fix manifest paths o...,"[bug, 1478457, wpt, pr, 12185, fix, manifest, ...","[bug, 1478457, wpt, pr, 12185, fix, manifest, ..."
4,96d519e918295f080f3963e132b8be0a2fbdf5c6,449443,Bug 449443 - Upgrade to SQLite 3.6.4 This chan...,176333,Bug 449443 Upgrade to SQLite 364 This changes...,"[bug, 449443, upgrade, to, sqlite, 364, this, ...","[bug, 449443, upgrade, sqlite, 364, changeset,..."
...,...,...,...,...,...,...,...
626296,5676846b3b61a99790169a2ed23abb76f38639b1,798951,Bug 798951 - Intermittent browser_webconsole_b...,1,Bug 798951 Intermittent browserwebconsolebug7...,"[bug, 798951, intermittent, browserwebconsoleb...","[bug, 798951, intermittent, browserwebconsoleb..."
626297,56901d3ac8fcd3219a65790edfa504a1272d2129,1389010,"Bug 1389010, part 7 - Call AllBordersSameWidth...",1,Bug 1389010 part 7 Call AllBordersSameWidth i...,"[bug, 1389010, part, 7, call, allborderssamewi...","[bug, 1389010, part, 7, call, allborderssamewi..."
626298,1ee861335f5f0c1e70925943d0520f5ceeacc956,326972,Use nsContentUtils helpers for some localizing...,1,Use nsContentUtils helpers for some localizing...,"[use, nscontentutils, helpers, for, some, loca...","[use, nscontentutils, helpers, localizing, nsg..."
626299,875f559c288abd157c6fd59d7deb2af1d2658426,809245,Bug 809245 - add preliminary touch support to ...,1,Bug 809245 add preliminary touch support to m...,"[bug, 809245, add, preliminary, touch, support...","[bug, 809245, add, preliminary, touch, support..."


In [17]:
from nltk.corpus import wordnet
import nltk
import re

nltk.download('wordnet')
nltk.download('omw-1.4')
wn = nltk.WordNetLemmatizer()

# Define a function to check if a word has a dictionary meaning. Input: word
def word_has_dictionary_meaning(word):
    synsets = wordnet.synsets(word)
    return len(synsets) > 0

# Input: array of words as token
def clean_numbers(token_txt):
    number_pattern = r'^-?\d+$'
    text = [word for word in token_txt if not re.match(number_pattern, word)]
    return text

# Define a function to lemmatize words. Input: array of words as token
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt]
    return text

# Apply the lemmatization function to the 'no_swds' column
df1['filtered_no_swds'] = df1['no_swds'].apply(lambda words: [word for word in words if word_has_dictionary_meaning(word)])

df1['lemmatized_no_swds'] = df1['filtered_no_swds'].apply(lambda x: lemmatization(x))

df1['clean_lemma']=df1['lemmatized_no_swds'].apply(lambda words: clean_numbers(words))

df1

[nltk_data] Downloading package wordnet to /home/taj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/taj/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,commit,bug_id,subject,churn,msg_clean,summary_tokenized,no_swds,filtered_no_swds,lemmatized_no_swds,clean_lemma
0,e413c3c4d11d93aca4b022ad9b4a1dea8b432af5,1542883,"Bug 1542883, Remove unused wpt manifest files,...",671424,Bug 1542883 Remove unused wpt manifest files r...,"[bug, 1542883, remove, unused, wpt, manifest, ...","[bug, 1542883, remove, unused, wpt, manifest, ...","[bug, remove, unused, manifest, files]","[bug, remove, unused, manifest, file]","[bug, remove, unused, manifest, file]"
1,a1b638d149015419baa713063f4a5f7874837986,1381842,Bug 1381842 - Update web-platform-tests to rev...,521797,Bug 1381842 Update webplatformtests to revisi...,"[bug, 1381842, update, webplatformtests, to, r...","[bug, 1381842, update, webplatformtests, revis...","[bug, update, revision]","[bug, update, revision]","[bug, update, revision]"
2,1bf284ae7396bf9f95440c85123382571a5c7385,1419296,Bug 1419296 - Update web-platform-tests to fa2...,399007,Bug 1419296 Update webplatformtests to fa27b7...,"[bug, 1419296, update, webplatformtests, to, f...","[bug, 1419296, update, webplatformtests, fa27b...","[bug, update]","[bug, update]","[bug, update]"
3,6892237b54272a2938d9f14824a9207563c2f7ae,1478457,Bug 1478457 [wpt PR 12185] - Fix manifest path...,383419,Bug 1478457 wpt PR 12185 Fix manifest paths o...,"[bug, 1478457, wpt, pr, 12185, fix, manifest, ...","[bug, 1478457, wpt, pr, 12185, fix, manifest, ...","[bug, pr, fix, manifest, paths, obtained, file...","[bug, pr, fix, manifest, path, obtained, file,...","[bug, pr, fix, manifest, path, obtained, file,..."
4,96d519e918295f080f3963e132b8be0a2fbdf5c6,449443,Bug 449443 - Upgrade to SQLite 3.6.4 This chan...,176333,Bug 449443 Upgrade to SQLite 364 This changes...,"[bug, 449443, upgrade, to, sqlite, 364, this, ...","[bug, 449443, upgrade, sqlite, 364, changeset,...","[bug, upgrade, actual, upgrade]","[bug, upgrade, actual, upgrade]","[bug, upgrade, actual, upgrade]"
...,...,...,...,...,...,...,...,...,...,...
626296,5676846b3b61a99790169a2ed23abb76f38639b1,798951,Bug 798951 - Intermittent browser_webconsole_b...,1,Bug 798951 Intermittent browserwebconsolebug7...,"[bug, 798951, intermittent, browserwebconsoleb...","[bug, 798951, intermittent, browserwebconsoleb...","[bug, intermittent, test, timed]","[bug, intermittent, test, timed]","[bug, intermittent, test, timed]"
626297,56901d3ac8fcd3219a65790edfa504a1272d2129,1389010,"Bug 1389010, part 7 - Call AllBordersSameWidth...",1,Bug 1389010 part 7 Call AllBordersSameWidth i...,"[bug, 1389010, part, 7, call, allborderssamewi...","[bug, 1389010, part, 7, call, allborderssamewi...","[bug, part, 7, call, constructor, store, resul...","[bug, part, 7, call, constructor, store, resul...","[bug, part, call, constructor, store, result, ..."
626298,1ee861335f5f0c1e70925943d0520f5ceeacc956,326972,Use nsContentUtils helpers for some localizing...,1,Use nsContentUtils helpers for some localizing...,"[use, nscontentutils, helpers, for, some, loca...","[use, nscontentutils, helpers, localizing, nsg...","[use, helpers, localizing, bug]","[use, helper, localizing, bug]","[use, helper, localizing, bug]"
626299,875f559c288abd157c6fd59d7deb2af1d2658426,809245,Bug 809245 - add preliminary touch support to ...,1,Bug 809245 add preliminary touch support to m...,"[bug, 809245, add, preliminary, touch, support...","[bug, 809245, add, preliminary, touch, support...","[bug, add, preliminary, touch, support, marion...","[bug, add, preliminary, touch, support, marion...","[bug, add, preliminary, touch, support, marion..."


In [8]:
# Check if words were lemmatized
df1['lemmatized_check'] = df1.apply(lambda row: any(original != lemmatized for original, lemmatized in zip(row['no_swds'], row['lemmatized_no_swds'])), axis=1)

# Filter rows where lemmatization was applied
lemmatized_rows = df1[df1['lemmatized_check']]

# Display the rows where lemmatization was applied
# print(lemmatized_rows[['no_swds', 'clean_lemma']])


In [9]:
df1['meaningful_lemm_str'] = df1['clean_lemma'].apply(lambda x: ' '.join(x))

In [10]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Define stop words to remove them from TFIDF matrix
stop = list(stopwords.words('english'))
stop.extend(['10','13','30','60','firefox','doesnt','using','user','use','new','zoomoutcur','x','xx','xxx','xxxx', 
             'xyz','äöü','类型','更新10101之后','没有找到支持的视频格式和','类','型','1st','2d','2nd','3d','3rd','aa','dd'])

# Update the word_to_drop list with the extended stopwords
word_to_drop = stop

# Create an instance of TfidfVectorizer with stop words
vectorizer = TfidfVectorizer(stop_words=word_to_drop, min_df=25, max_features=2000)

# Fit and transform your text data using the vectorizer
tfidf_matrix = vectorizer.fit_transform(df1['meaningful_lemm_str'])

# Apply threshold on TF-IDF values
min_threshold = 0.05  # example value
max_threshold = 0.8  # example value

# Set values below min_threshold to 0 and values above max_threshold to max_threshold
tfidf_matrix = tfidf_matrix.todense()  # Convert sparse matrix to dense
tfidf_matrix[tfidf_matrix < min_threshold] = 0
tfidf_matrix[tfidf_matrix > max_threshold] = max_threshold

# Get the feature names (i.e., words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Iterate over feature names and their corresponding TF-IDF scores for each document
#for i in range(len(df1)):
#    print(f"Document {i+1}:")
#    for j, word in enumerate(feature_names):
#        tfidf_score = tfidf_matrix[i, j]
#        if tfidf_score > 0.3:
#            print(f"{word}: {tfidf_score:.4f}")

In [1]:
#elbow curve to determine the number of clusters

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Define a range of cluster numbers to try
num_clusters_range = range(1, 11)  # You can adjust the range as needed

# Initialize a list to store the inertia values
inertia_values = []

# Fit KMeans for each value of k and calculate inertia
for k in num_clusters_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(np.asarray(tfidf_matrix))
    inertia_values.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(num_clusters_range, inertia_values, marker='o')
plt.title('Elbow Curve for KMeans')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(num_clusters_range)
plt.show()

NameError: name 'np' is not defined

In [ ]:
from sklearn.cluster import KMeans

# Define the number of clusters
num_clusters = 3

# Create an instance of KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit the KMeans model to your TF-IDF matrix
kmeans.fit(np.asarray(tfidf_matrix))

# Get cluster labels for each document
cluster_labels = kmeans.labels_

# Print cluster assignments for the first 100 documents (for illustration)
for i, label in enumerate(cluster_labels[:100]):  
    print(f"Document {i+1} belongs to cluster {label}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Reduce dimensionality using PCA
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(np.asarray(tfidf_matrix))

# Create a scatter plot of the clusters
plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=cluster_labels, cmap='viridis')
plt.title('K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
n_clusters = 3

# create KMeans clustering object
kmeans = KMeans(n_clusters=n_clusters, random_state=49)

# fit KMeans clustering model to the tfidf_matrix
kmeans.fit(np.asarray(tfidf_matrix))

# get cluster labels for each row in the tfidf_matrix
df1['cluster_labels'] = kmeans.labels_

# print number of rows in each cluster
unique, counts = np.unique(cluster_labels, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Create a dictionary to store the documents in each cluster
cluster_docs = {}
for i, label in enumerate(cluster_labels):
    if label not in cluster_docs:
        cluster_docs[label] = []
    cluster_docs[label].append(i)

# Create word cloud for each cluster
for label, doc_indices in cluster_docs.items():
    # Retrieve the words from the TfidfVectorizer object
    cluster_words = " ".join(feature_names[i] for i in tfidf_matrix[doc_indices].nonzero()[1])

    # Check if cluster contains at least one document with words
    if cluster_words:
        # Create a WordCloud object
        wordcloud = WordCloud(width=800, height=400, max_words=150, background_color='white').generate(cluster_words)

        # Plot the WordCloud
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Cluster {label} Word Cloud')
        plt.axis('off')
        plt.show()
    else:
        print(f"No documents with words in Cluster {label}")

